# 🏪 서울시 편의점 매출 결정요인 종합 분석

---

## 📋 분석 개요

**연구 질문**: 서울시 편의점 매출을 결정하는 핵심 요인은 무엇인가?

**연구 가설**:
- H1: 유동인구가 많을수록 매출이 높다
- H2: 점포수(밀집도)가 높을수록 매출이 높다
- H3: 상권유형(골목/발달/전통시장/관광특구)에 따라 매출 차이가 있다

**분석 기간**: 2022년 1분기 ~ 2025년 3분기 (15개 분기)

---

## 📑 목차

1. **데이터 전처리 및 EDA**
2. **기초 통계분석 (상관분석, t-검정)**
3. **회귀분석 (OLS, VIF, 잔차분석)**
4. **ANOVA 및 유의성 검증**
5. **추가 분석 기법 (Ridge/Lasso, Bootstrap)**
6. **머신러닝 모델링 (Random Forest, Gradient Boosting, XGBoost)**
7. **What-if 시나리오 분석**
8. **상권유형 전환 효과 분석**
9. **종합 예측 및 최종 결론**
10. **핵심 인사이트 및 전략적 제언**

---

# 1. 데이터 전처리 및 EDA

In [1]:
# 필수 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway, ttest_ind, shapiro, levene, pearsonr, spearmanr
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 출력 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

print("✅ 라이브러리 로드 완료")

✅ 라이브러리 로드 완료


In [2]:
# 데이터 로드
df = pd.read_csv('분석데이터.csv', encoding='utf-8-sig')

print("="*70)
print("📊 데이터 기본 정보")
print("="*70)
print(f"총 데이터 수: {len(df):,}건")
print(f"컬럼 수: {len(df.columns)}개")
print(f"\n📌 컬럼 목록:")
for col in df.columns:
    print(f"  - {col}: {df[col].dtype}")

📊 데이터 기본 정보
총 데이터 수: 6,097건
컬럼 수: 10개

📌 컬럼 목록:
  - 기준_년분기_코드: int64
  - 행정동_코드: int64
  - 행정동_코드_명: object
  - 당월_매출_금액: float64
  - 당월_매출_건수: int64
  - 점포_수: int64
  - 총_유동인구_수: int64
  - 주요_상권유형: object
  - 연도: int64
  - 분기: int64


In [3]:
# 기초 통계량
print("\n" + "="*70)
print("📈 주요 변수 기술통계")
print("="*70)

numeric_cols = ['당월_매출_금액', '당월_매출_건수', '점포_수', '총_유동인구_수']
stats_df = df[numeric_cols].describe()

# 억원 단위로 변환하여 출력
print(f"\n[매출 통계]")
print(f"  평균: {df['당월_매출_금액'].mean()/1e8:.2f}억원")
print(f"  중앙값: {df['당월_매출_금액'].median()/1e8:.2f}억원")
print(f"  표준편차: {df['당월_매출_금액'].std()/1e8:.2f}억원")
print(f"  최소: {df['당월_매출_금액'].min()/1e8:.2f}억원")
print(f"  최대: {df['당월_매출_금액'].max()/1e8:.2f}억원")

print(f"\n[점포 수 통계]")
print(f"  평균: {df['점포_수'].mean():.1f}개")
print(f"  중앙값: {df['점포_수'].median():.1f}개")
print(f"  최소: {df['점포_수'].min():.0f}개")
print(f"  최대: {df['점포_수'].max():.0f}개")

print(f"\n[유동인구 통계]")
print(f"  평균: {df['총_유동인구_수'].mean()/1e6:.2f}백만명")
print(f"  중앙값: {df['총_유동인구_수'].median()/1e6:.2f}백만명")
print(f"  최소: {df['총_유동인구_수'].min()/1e6:.2f}백만명")
print(f"  최대: {df['총_유동인구_수'].max()/1e6:.2f}백만명")


📈 주요 변수 기술통계

[매출 통계]
  평균: 27.11억원
  중앙값: 17.19억원
  표준편차: 32.01억원
  최소: 0.00억원
  최대: 351.91억원

[점포 수 통계]
  평균: 6.2개
  중앙값: 5.0개
  최소: 0개
  최대: 51개

[유동인구 통계]
  평균: 5.62백만명
  중앙값: 5.26백만명
  최소: 0.02백만명
  최대: 22.33백만명


In [4]:
# 상권유형별 분포
print("\n" + "="*70)
print("🏬 상권유형별 분포")
print("="*70)

area_dist = df.groupby('주요_상권유형').agg({
    '행정동_코드': 'count',
    '당월_매출_금액': ['mean', 'median', 'std'],
    '점포_수': 'mean',
    '총_유동인구_수': 'mean'
}).round(2)

area_dist.columns = ['건수', '평균매출', '중앙매출', '표준편차', '평균점포수', '평균유동인구']

print(f"\n{'상권유형':<12} {'건수':>8} {'평균매출(억)':>12} {'중앙매출(억)':>12} {'평균점포수':>10} {'평균유동인구(만)':>14}")
print("-"*70)
for idx, row in area_dist.iterrows():
    print(f"{idx:<12} {row['건수']:>8.0f} {row['평균매출']/1e8:>12.2f} {row['중앙매출']/1e8:>12.2f} {row['평균점포수']:>10.1f} {row['평균유동인구']/1e4:>14.0f}")


🏬 상권유형별 분포

상권유형               건수      평균매출(억)      중앙매출(억)      평균점포수      평균유동인구(만)
----------------------------------------------------------------------
골목상권             5137        24.91        16.80        6.0            583
관광특구               90        82.61        84.61       13.5            611
미분류               339        13.28         6.22        3.4            279
발달상권              300        62.63        48.43       11.9            543
전통시장              231        28.53        19.56        4.2            519


In [5]:
# 시각화: 상권유형별 매출 분포

# 색상
colors = ['#4E79A7', '#F28E2B', '#59A14F', '#E15759', '#76B7B2']

# 상권유형 순서 (중앙값 기준)
order = (
    df.groupby('주요_상권유형')['당월_매출_금액']
    .median()
    .sort_values()
    .index
)

# 서브플롯 생성
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        "상권유형별 매출 분포",
        "점포 수와 매출의 관계",
        "유동인구와 매출의 관계",
        "상권유형별 평균 매출"
    ]
)

# --------------------------------------------------
# 1. 상권유형별 매출 박스플롯
# --------------------------------------------------
for i, t in enumerate(order):
    fig.add_trace(
        go.Box(
            y=df[df['주요_상권유형'] == t]['당월_매출_금액'] / 1e8,
            name=t,  # 범례/hover 한글
            marker_color=colors[i % len(colors)],
            opacity=0.7,
            hovertemplate=(
                "상권유형: %{name}<br>"
                "매출액: %{y:.2f} 억원<br>"
                "<extra></extra>"
            )
        ),
        row=1, col=1
    )

fig.update_yaxes(title_text="매출액 (억원)", row=1, col=1)

# --------------------------------------------------
# 2. 점포 수 vs 매출 산점도
# --------------------------------------------------
for i, t in enumerate(order):
    dff = df[df['주요_상권유형'] == t]
    fig.add_trace(
        go.Scatter(
            x=dff['점포_수'],
            y=dff['당월_매출_금액'] / 1e8,
            mode='markers',
            name=t,
            marker=dict(
                size=6,
                color=colors[i % len(colors)],
                opacity=0.5
            ),
            hovertemplate=(
                "상권유형: %{name}<br>"
                "점포 수: %{x} 개<br>"
                "매출액: %{y:.2f} 억원<br>"
                "<extra></extra>"
            ),
            showlegend=False
        ),
        row=1, col=2
    )

fig.update_xaxes(title_text="점포 수 (개)", row=1, col=2)
fig.update_yaxes(title_text="매출액 (억원)", row=1, col=2)

# --------------------------------------------------
# 3. 유동인구 vs 매출 산점도
# --------------------------------------------------
for i, t in enumerate(order):
    dff = df[df['주요_상권유형'] == t]
    fig.add_trace(
        go.Scatter(
            x=dff['총_유동인구_수'] / 1e6,
            y=dff['당월_매출_금액'] / 1e8,
            mode='markers',
            name=t,
            marker=dict(
                size=6,
                color=colors[i % len(colors)],
                opacity=0.5
            ),
            hovertemplate=(
                "상권유형: %{name}<br>"
                "유동인구: %{x:.2f} 백만 명<br>"
                "매출액: %{y:.2f} 억원<br>"
                "<extra></extra>"
            ),
            showlegend=False
        ),
        row=2, col=1
    )

fig.update_xaxes(title_text="유동인구 (백만 명)", row=2, col=1)
fig.update_yaxes(title_text="매출액 (억원)", row=2, col=1)

# --------------------------------------------------
# 4. 상권유형별 평균 매출
# --------------------------------------------------
mean_sales = (
    df.groupby('주요_상권유형')['당월_매출_금액']
    .mean()
    .sort_values()
)

fig.add_trace(
    go.Bar(
        x=mean_sales.values / 1e8,
        y=mean_sales.index,
        orientation='h',
        marker_color=colors[:len(mean_sales)],
        text=[f"{v/1e8:.1f} 억원" for v in mean_sales.values],
        textposition='outside',
        hovertemplate=(
            "상권유형: %{y}<br>"
            "평균 매출액: %{x:.2f} 억원<br>"
            "<extra></extra>"
        )
    ),
    row=2, col=2
)

fig.update_xaxes(title_text="평균 매출액 (억원)", row=2, col=2)

# --------------------------------------------------
# 전체 레이아웃
# --------------------------------------------------
fig.update_layout(
    height=900,
    width=1200,
    title_text="서울시 상권유형별 편의점 매출 EDA",
    title_x=0.5,
    showlegend=False
)

# 저장
fig.write_image("01_EDA_overview.png", scale=2)

fig.show()

print("✅ EDA 시각화 저장 완료: 01_EDA_overview.png")

✅ EDA 시각화 저장 완료: 01_EDA_overview.png


---

# 2. 기초 통계분석 (상관분석, t-검정)

In [6]:
# 상관분석
print("="*70)
print("📊 상관분석 (Correlation Analysis)")
print("="*70)

# Pearson 상관계수
corr_vars = ['당월_매출_금액', '점포_수', '총_유동인구_수', '당월_매출_건수']
corr_matrix = df[corr_vars].corr()

print("\n[Pearson 상관계수 행렬]")
print(corr_matrix.round(4))

# 개별 상관관계 검정
print("\n[매출과 주요 변수 간 상관관계 검정]")
print("-"*60)

# 점포수 - 매출
r1, p1 = pearsonr(df['점포_수'], df['당월_매출_금액'])
print(f"점포수 vs 매출:     r = {r1:.4f}, p-value = {p1:.2e} {'***' if p1<0.001 else '**' if p1<0.01 else '*' if p1<0.05 else ''}")

# 유동인구 - 매출
r2, p2 = pearsonr(df['총_유동인구_수'], df['당월_매출_금액'])
print(f"유동인구 vs 매출:   r = {r2:.4f}, p-value = {p2:.2e} {'***' if p2<0.001 else '**' if p2<0.01 else '*' if p2<0.05 else ''}")

# 점포수 - 유동인구
r3, p3 = pearsonr(df['점포_수'], df['총_유동인구_수'])
print(f"점포수 vs 유동인구: r = {r3:.4f}, p-value = {p3:.2e} {'***' if p3<0.001 else '**' if p3<0.01 else '*' if p3<0.05 else ''}")

print("\n※ 유의수준: *** p<0.001, ** p<0.01, * p<0.05")

📊 상관분석 (Correlation Analysis)

[Pearson 상관계수 행렬]
          당월_매출_금액  점포_수  총_유동인구_수  당월_매출_건수
당월_매출_금액      1.00  0.81      0.50      0.99
점포_수          0.81  1.00      0.48      0.79
총_유동인구_수      0.50  0.48      1.00      0.48
당월_매출_건수      0.99  0.79      0.48      1.00

[매출과 주요 변수 간 상관관계 검정]
------------------------------------------------------------
점포수 vs 매출:     r = 0.8065, p-value = 0.00e+00 ***
유동인구 vs 매출:   r = 0.4991, p-value = 0.00e+00 ***
점포수 vs 유동인구: r = 0.4848, p-value = 0.00e+00 ***

※ 유의수준: *** p<0.001, ** p<0.01, * p<0.05


In [7]:
# 상관관계 히트맵
num_df = df.select_dtypes(include=[np.number])

corr_matrix = num_df.corr()

fig = px.imshow(
    corr_matrix,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    origin='lower',
    title='변수 간 상관관계 히트맵'
)

fig.show()

In [8]:
# 독립표본 t-검정: 발달상권 vs 골목상권
print("\n" + "="*70)
print("📊 독립표본 t-검정 (발달상권 vs 골목상권)")
print("="*70)

developed = df[df['주요_상권유형'] == '발달상권']['당월_매출_금액']
alley = df[df['주요_상권유형'] == '골목상권']['당월_매출_금액']

# 등분산 검정 (Levene's test)
levene_stat, levene_p = levene(developed, alley)
print(f"\n[등분산 검정 (Levene)]")
print(f"  통계량: {levene_stat:.4f}")
print(f"  p-value: {levene_p:.4e}")
print(f"  결과: {'등분산 가정 충족' if levene_p > 0.05 else '등분산 가정 불충족 → Welch t-test 적용'}")

# t-검정 (Welch's t-test: 등분산 가정 X)
t_stat, t_p = ttest_ind(developed, alley, equal_var=False)
print(f"\n[Welch's t-검정]")
print(f"  t-통계량: {t_stat:.4f}")
print(f"  p-value: {t_p:.4e}")
print(f"  결론: {'유의미한 차이 있음' if t_p < 0.05 else '유의미한 차이 없음'} (α=0.05)")

print(f"\n[기술통계]")
print(f"  발달상권 평균: {developed.mean()/1e8:.2f}억원 (n={len(developed)})")
print(f"  골목상권 평균: {alley.mean()/1e8:.2f}억원 (n={len(alley)})")
print(f"  차이: {(developed.mean() - alley.mean())/1e8:.2f}억원")


📊 독립표본 t-검정 (발달상권 vs 골목상권)

[등분산 검정 (Levene)]
  통계량: 161.9868
  p-value: 1.3802e-36
  결과: 등분산 가정 불충족 → Welch t-test 적용

[Welch's t-검정]
  t-통계량: 13.1045
  p-value: 1.4839e-31
  결론: 유의미한 차이 있음 (α=0.05)

[기술통계]
  발달상권 평균: 62.63억원 (n=300)
  골목상권 평균: 24.91억원 (n=5137)
  차이: 37.72억원


---

# 3. 회귀분석 (OLS, VIF, 잔차분석)

In [9]:
# 변수 준비
print("="*70)
print("📊 다중선형회귀분석 (Multiple Linear Regression)")
print("="*70)

# 더미변수 생성 (기준: 골목상권)
df['발달상권'] = (df['주요_상권유형'] == '발달상권').astype(int)
df['전통시장'] = (df['주요_상권유형'] == '전통시장').astype(int)
df['관광특구'] = (df['주요_상권유형'] == '관광특구').astype(int)
df['미분류'] = (df['주요_상권유형'] == '미분류').astype(int)

# 독립변수, 종속변수 설정
feature_cols = ['점포_수', '총_유동인구_수', '발달상권', '전통시장', '관광특구', '미분류']
X = df[feature_cols].values
y = df['당월_매출_금액'].values

# 상수항 추가
X_with_const = np.column_stack([np.ones(len(X)), X])

print(f"\n[모델 설계]")
print(f"종속변수: 당월_매출_금액")
print(f"독립변수: {feature_cols}")
print(f"기준범주: 골목상권")
print(f"\n회귀식: 매출 = β₀ + β₁(점포수) + β₂(유동인구) + β₃(발달상권) + β₄(전통시장) + β₅(관광특구) + β₆(미분류) + ε")

📊 다중선형회귀분석 (Multiple Linear Regression)

[모델 설계]
종속변수: 당월_매출_금액
독립변수: ['점포_수', '총_유동인구_수', '발달상권', '전통시장', '관광특구', '미분류']
기준범주: 골목상권

회귀식: 매출 = β₀ + β₁(점포수) + β₂(유동인구) + β₃(발달상권) + β₄(전통시장) + β₅(관광특구) + β₆(미분류) + ε


In [10]:
# VIF (다중공선성) 계산
print("\n" + "="*70)
print("📊 다중공선성 검정 (VIF - Variance Inflation Factor)")
print("="*70)
print("\n기준: VIF < 5 양호, 5~10 주의, > 10 심각")
print("-"*50)

def calculate_vif(X, feature_names):
    """VIF 계산 함수"""
    vif_data = []
    for i in range(X.shape[1]):
        # i번째 변수를 종속변수로, 나머지를 독립변수로
        y_temp = X[:, i]
        X_temp = np.delete(X, i, axis=1)
        
        # R² 계산
        model = LinearRegression()
        model.fit(X_temp, y_temp)
        r2 = model.score(X_temp, y_temp)
        
        # VIF 계산
        vif = 1 / (1 - r2) if r2 < 1 else float('inf')
        
        status = '양호' if vif < 5 else ('주의' if vif < 10 else '심각')
        vif_data.append({'변수': feature_names[i], 'VIF': vif, '상태': status})
    
    return pd.DataFrame(vif_data)

vif_df = calculate_vif(X, feature_cols)
for _, row in vif_df.iterrows():
    print(f"  {row['변수']:<15}: {row['VIF']:.4f} ({row['상태']})")

max_vif = vif_df['VIF'].max()
print(f"\n최대 VIF: {max_vif:.4f} → {'다중공선성 문제 없음' if max_vif < 5 else '다중공선성 주의 필요'}")


📊 다중공선성 검정 (VIF - Variance Inflation Factor)

기준: VIF < 5 양호, 5~10 주의, > 10 심각
--------------------------------------------------
  점포_수           : 1.4727 (양호)
  총_유동인구_수       : 1.4061 (양호)
  발달상권           : 1.0962 (양호)
  전통시장           : 1.0097 (양호)
  관광특구           : 1.0393 (양호)
  미분류            : 1.0648 (양호)

최대 VIF: 1.4727 → 다중공선성 문제 없음


In [11]:
# OLS 회귀분석 (수동 구현)
print("\n" + "="*70)
print("📊 OLS 회귀분석 결과")
print("="*70)

# OLS 추정: β = (X'X)^(-1)X'y
beta = np.linalg.lstsq(X_with_const, y, rcond=None)[0]

# 예측값 및 잔차
y_pred = X_with_const @ beta
residuals = y - y_pred

# 모델 통계량
n = len(y)
k = X_with_const.shape[1] - 1  # 상수항 제외

# SST, SSR, SSE
SST = np.sum((y - np.mean(y))**2)
SSR = np.sum((y_pred - np.mean(y))**2)
SSE = np.sum(residuals**2)

# R², Adjusted R²
R2 = 1 - SSE/SST
R2_adj = 1 - (1-R2)*(n-1)/(n-k-1)

# MSE, RMSE
MSE = SSE / (n - k - 1)
RMSE = np.sqrt(MSE)

# F-통계량
F_stat = (SSR/k) / (SSE/(n-k-1))
F_pvalue = 1 - stats.f.cdf(F_stat, k, n-k-1)

print(f"\n[모델 적합도]")
print(f"  R²:         {R2:.4f} (매출 변동의 {R2*100:.1f}% 설명)")
print(f"  Adjusted R²: {R2_adj:.4f}")
print(f"  RMSE:       {RMSE/1e8:.2f}억원")
print(f"  F-통계량:    {F_stat:.2f}")
print(f"  F p-value:  {F_pvalue:.2e} {'***' if F_pvalue < 0.001 else ''}")


📊 OLS 회귀분석 결과

[모델 적합도]
  R²:         0.6881 (매출 변동의 68.8% 설명)
  Adjusted R²: 0.6878
  RMSE:       17.89억원
  F-통계량:    2238.95
  F p-value:  0.00e+00 ***


In [12]:
# 회귀계수 및 유의성 검정
print("\n[회귀계수 및 유의성 검정]")
print("="*80)

# 표준오차 계산
var_beta = MSE * np.linalg.inv(X_with_const.T @ X_with_const)
se_beta = np.sqrt(np.diag(var_beta))

# t-통계량 및 p-value
t_stats = beta / se_beta
p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), n-k-1))

# 95% 신뢰구간
t_critical = stats.t.ppf(0.975, n-k-1)
ci_lower = beta - t_critical * se_beta
ci_upper = beta + t_critical * se_beta

coef_names = ['(상수항)'] + feature_cols
print(f"{'변수':<15} {'계수':>18} {'표준오차':>15} {'t-통계량':>12} {'p-value':>12} {'유의'}")
print("-"*80)

for i, name in enumerate(coef_names):
    sig = '***' if p_values[i] < 0.001 else ('**' if p_values[i] < 0.01 else ('*' if p_values[i] < 0.05 else ''))
    print(f"{name:<15} {beta[i]:>18,.0f} {se_beta[i]:>15,.0f} {t_stats[i]:>12.3f} {p_values[i]:>12.2e} {sig:>4}")

print("-"*80)
print("※ 유의수준: *** p<0.001, ** p<0.01, * p<0.05")


[회귀계수 및 유의성 검정]
변수                              계수            표준오차        t-통계량      p-value 유의
--------------------------------------------------------------------------------
(상수항)                 -973,220,686      52,694,731      -18.469     0.00e+00  ***
점포_수                   399,192,392       5,012,217       79.644     0.00e+00  ***
총_유동인구_수                       186               9       20.318     0.00e+00  ***
발달상권                 1,471,873,945     110,897,826       13.272     0.00e+00  ***
전통시장                 1,194,138,984     120,577,832        9.903     0.00e+00  ***
관광특구                 2,719,155,654     193,674,734       14.040     0.00e+00  ***
미분류                    439,417,000     103,168,676        4.259     2.08e-05  ***
--------------------------------------------------------------------------------
※ 유의수준: *** p<0.001, ** p<0.01, * p<0.05


In [13]:
# 회귀계수 해석
print("\n" + "="*70)
print("📝 회귀계수 해석")
print("="*70)

print(f"""
[계수 해석 - 실무적 의미]

1. 점포수 (β={beta[1]/1e8:.2f}억원)
   → 점포 1개 증가 시, 매출이 약 {beta[1]/1e8:.2f}억원 증가
   → 가장 강력한 매출 결정 요인

2. 유동인구 (β={beta[2]*1e6/1e8:.2f}억원/백만명)
   → 유동인구 100만명 증가 시, 매출이 약 {beta[2]*1e6/1e8:.2f}억원 증가
   → 통계적으로 유의하나 효과 크기는 상대적으로 작음

3. 상권유형 효과 (골목상권 대비)
   → 발달상권: +{beta[3]/1e8:.2f}억원
   → 전통시장: +{beta[4]/1e8:.2f}억원
   → 관광특구: +{beta[5]/1e8:.2f}억원 (가장 높음)
   → 미분류:   +{beta[6]/1e8:.2f}억원
""")


📝 회귀계수 해석

[계수 해석 - 실무적 의미]

1. 점포수 (β=3.99억원)
   → 점포 1개 증가 시, 매출이 약 3.99억원 증가
   → 가장 강력한 매출 결정 요인

2. 유동인구 (β=1.86억원/백만명)
   → 유동인구 100만명 증가 시, 매출이 약 1.86억원 증가
   → 통계적으로 유의하나 효과 크기는 상대적으로 작음

3. 상권유형 효과 (골목상권 대비)
   → 발달상권: +14.72억원
   → 전통시장: +11.94억원
   → 관광특구: +27.19억원 (가장 높음)
   → 미분류:   +4.39억원



In [14]:
# 잔차분석
print("\n" + "="*70)
print("📊 잔차분석 (Residual Analysis)")
print("="*70)

# 1. 정규성 검정 (Shapiro-Wilk) - 표본추출
sample_size = min(5000, len(residuals))
residual_sample = np.random.choice(residuals, sample_size, replace=False)
shapiro_stat, shapiro_p = shapiro(residual_sample)

print(f"\n[1. 정규성 검정 (Shapiro-Wilk)]")
print(f"  통계량: {shapiro_stat:.4f}")
print(f"  p-value: {shapiro_p:.4e}")
print(f"  결과: {'정규성 가정 충족' if shapiro_p > 0.05 else '정규성 가정 불충족'}")

# 2. 자기상관 검정 (Durbin-Watson)
dw_stat = np.sum(np.diff(residuals)**2) / np.sum(residuals**2)
print(f"\n[2. 자기상관 검정 (Durbin-Watson)]")
print(f"  DW 통계량: {dw_stat:.4f}")
print(f"  해석: {'자기상관 없음' if 1.5 < dw_stat < 2.5 else '자기상관 있을 수 있음'}")
print(f"  (기준: 2에 가까울수록 자기상관 없음)")

# 3. 이분산성 검정 (Breusch-Pagan 간이 검정)
# 잔차제곱을 예측값에 회귀
resid_sq = residuals**2
bp_model = LinearRegression()
bp_model.fit(y_pred.reshape(-1, 1), resid_sq)
bp_pred = bp_model.predict(y_pred.reshape(-1, 1))
bp_r2 = bp_model.score(y_pred.reshape(-1, 1), resid_sq)
bp_stat = n * bp_r2
bp_pvalue = 1 - stats.chi2.cdf(bp_stat, 1)

print(f"\n[3. 이분산성 검정 (간이 Breusch-Pagan)]")
print(f"  BP 통계량: {bp_stat:.4f}")
print(f"  p-value: {bp_pvalue:.4e}")
print(f"  결과: {'등분산 가정 충족' if bp_pvalue > 0.05 else '이분산성 존재 → Robust SE 권장'}")


📊 잔차분석 (Residual Analysis)

[1. 정규성 검정 (Shapiro-Wilk)]
  통계량: 0.8958
  p-value: 6.6889e-50
  결과: 정규성 가정 불충족

[2. 자기상관 검정 (Durbin-Watson)]
  DW 통계량: 1.9079
  해석: 자기상관 없음
  (기준: 2에 가까울수록 자기상관 없음)

[3. 이분산성 검정 (간이 Breusch-Pagan)]
  BP 통계량: 792.9094
  p-value: 0.0000e+00
  결과: 이분산성 존재 → Robust SE 권장


In [15]:
# 잔차 시각화
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        "잔차 vs 적합값",
        "잔차 분포",
        "Q-Q Plot",
        "Scale-Location"
    ]
)

# 1. 잔차 vs 적합값
fig.add_trace(
    go.Scatter(
        x=y_pred / 1e8,
        y=residuals / 1e8,
        mode='markers',
        opacity=0.4
    ),
    row=1, col=1
)

# 2. 잔차 분포
fig.add_trace(
    go.Histogram(
        x=residuals / 1e8,
        nbinsx=40
    ),
    row=1, col=2
)

# 3. Q-Q Plot
sorted_resid = np.sort(residuals)
theoretical_q = stats.norm.ppf(
    np.linspace(0.01, 0.99, len(sorted_resid))
)

fig.add_trace(
    go.Scatter(
        x=theoretical_q,
        y=sorted_resid,
        mode='markers'
    ),
    row=2, col=1
)

# 4. Scale-Location
fig.add_trace(
    go.Scatter(
        x=y_pred / 1e8,
        y=np.sqrt(np.abs(residuals)),
        mode='markers',
        opacity=0.4
    ),
    row=2, col=2
)

fig.update_layout(
    height=800,
    title_text='회귀모형 잔차 진단'
)

fig.show()

---

# 4. ANOVA 및 유의성 검증

In [16]:
# One-way ANOVA: 상권유형별 매출 차이 검정
print("="*70)
print("📊 일원분산분석 (One-way ANOVA)")
print("="*70)
print("\n귀무가설(H0): 모든 상권유형의 평균 매출은 동일하다")
print("대립가설(H1): 적어도 하나의 상권유형의 평균 매출이 다르다")

# 상권유형별 그룹 분리
groups = [df[df['주요_상권유형'] == t]['당월_매출_금액'].values 
          for t in df['주요_상권유형'].unique()]

# ANOVA 검정
f_stat, anova_p = f_oneway(*groups)

print(f"\n[ANOVA 결과]")
print(f"  F-통계량: {f_stat:.4f}")
print(f"  p-value: {anova_p:.4e}")
print(f"  결론: {'귀무가설 기각 - 상권유형별 매출 차이 유의함' if anova_p < 0.05 else '귀무가설 채택'}")

📊 일원분산분석 (One-way ANOVA)

귀무가설(H0): 모든 상권유형의 평균 매출은 동일하다
대립가설(H1): 적어도 하나의 상권유형의 평균 매출이 다르다

[ANOVA 결과]
  F-통계량: 206.4593
  p-value: 2.4593e-166
  결론: 귀무가설 기각 - 상권유형별 매출 차이 유의함


In [17]:
# 상권유형별 기술통계
print("\n[상권유형별 매출 기술통계]")
print("-"*70)
print(f"{'상권유형':<12} {'n':>6} {'평균(억)':>12} {'표준편차(억)':>12} {'중앙값(억)':>12}")
print("-"*70)

for t in df['주요_상권유형'].unique():
    subset = df[df['주요_상권유형'] == t]['당월_매출_금액']
    print(f"{t:<12} {len(subset):>6} {subset.mean()/1e8:>12.2f} {subset.std()/1e8:>12.2f} {subset.median()/1e8:>12.2f}")


[상권유형별 매출 기술통계]
----------------------------------------------------------------------
상권유형              n        평균(억)      표준편차(억)       중앙값(억)
----------------------------------------------------------------------
골목상권           5137        24.91        29.69        16.80
발달상권            300        62.63        49.33        48.43
관광특구             90        82.61        18.72        84.61
전통시장            231        28.53        23.74        19.56
미분류             339        13.28        16.08         6.22


In [18]:
# 사후검정 (Tukey HSD 대안: 다중 t-검정 with Bonferroni 보정)
print("\n" + "="*70)
print("📊 사후검정 (Pairwise t-tests with Bonferroni correction)")
print("="*70)

area_types = df['주요_상권유형'].unique()
n_comparisons = len(area_types) * (len(area_types) - 1) // 2
bonferroni_alpha = 0.05 / n_comparisons

print(f"\nBonferroni 보정 유의수준: α = {bonferroni_alpha:.4f} (원래 0.05 / {n_comparisons}개 비교)")
print("\n" + "-"*80)
print(f"{'비교 그룹':<30} {'t-통계량':>12} {'p-value':>15} {'유의':>8} {'차이(억)':>12}")
print("-"*80)

for i, type1 in enumerate(area_types):
    for j, type2 in enumerate(area_types):
        if i < j:
            g1 = df[df['주요_상권유형'] == type1]['당월_매출_금액']
            g2 = df[df['주요_상권유형'] == type2]['당월_매출_금액']
            t_stat, p_val = ttest_ind(g1, g2, equal_var=False)
            sig = '*' if p_val < bonferroni_alpha else ''
            diff = (g1.mean() - g2.mean()) / 1e8
            comparison = f"{type1} vs {type2}"
            print(f"{comparison:<30} {t_stat:>12.3f} {p_val:>15.4e} {sig:>8} {diff:>+12.2f}")

print("-"*80)
print(f"※ * = p < {bonferroni_alpha:.4f} (Bonferroni 보정 후 유의)")


📊 사후검정 (Pairwise t-tests with Bonferroni correction)

Bonferroni 보정 유의수준: α = 0.0050 (원래 0.05 / 10개 비교)

--------------------------------------------------------------------------------
비교 그룹                                 t-통계량         p-value       유의        차이(억)
--------------------------------------------------------------------------------
골목상권 vs 발달상권                        -13.105      1.4839e-31        *       -37.72
골목상권 vs 관광특구                        -28.622      4.2531e-49        *       -57.70
골목상권 vs 전통시장                         -2.238      2.6045e-02                 -3.62
골목상권 vs 미분류                          12.033      1.6725e-29        *       +11.63
발달상권 vs 관광특구                         -5.767      1.7047e-08        *       -19.98
발달상권 vs 전통시장                         10.498      3.2039e-23        *       +34.10
발달상권 vs 미분류                          16.565      4.5765e-46        *       +49.35
관광특구 vs 전통시장                         21.493      2.3410e-54        *       +

In [19]:
# 효과크기 계산 (Eta-squared)
print("\n" + "="*70)
print("📊 효과크기 (Effect Size)")
print("="*70)

# SSB (Between-group SS), SSW (Within-group SS)
grand_mean = df['당월_매출_금액'].mean()
SSB = sum(len(df[df['주요_상권유형'] == t]) * (df[df['주요_상권유형'] == t]['당월_매출_금액'].mean() - grand_mean)**2 
          for t in area_types)
SSW = sum(((df[df['주요_상권유형'] == t]['당월_매출_금액'] - df[df['주요_상권유형'] == t]['당월_매출_금액'].mean())**2).sum() 
          for t in area_types)
SST_anova = SSB + SSW

# Eta-squared
eta_squared = SSB / SST_anova

# Omega-squared (더 보수적인 추정)
k_groups = len(area_types)
MSW = SSW / (n - k_groups)
omega_squared = (SSB - (k_groups - 1) * MSW) / (SST_anova + MSW)

print(f"\n[효과크기 해석 기준] (Cohen, 1988)")
print(f"  η² < 0.01: 작은 효과")
print(f"  η² = 0.06: 중간 효과")
print(f"  η² > 0.14: 큰 효과")

print(f"\n[분석 결과]")
print(f"  Eta-squared (η²): {eta_squared:.4f}")
print(f"  Omega-squared (ω²): {omega_squared:.4f}")

effect_size = '큰 효과' if eta_squared > 0.14 else ('중간 효과' if eta_squared > 0.06 else '작은 효과')
print(f"  해석: 상권유형이 매출 변동의 {eta_squared*100:.1f}%를 설명 → {effect_size}")


📊 효과크기 (Effect Size)

[효과크기 해석 기준] (Cohen, 1988)
  η² < 0.01: 작은 효과
  η² = 0.06: 중간 효과
  η² > 0.14: 큰 효과

[분석 결과]
  Eta-squared (η²): 0.1194
  Omega-squared (ω²): 0.1188
  해석: 상권유형이 매출 변동의 11.9%를 설명 → 중간 효과


---

# 5. 추가 분석 기법 (Ridge/Lasso, Bootstrap)

In [20]:
# 정규화 회귀분석 (Ridge, Lasso, ElasticNet)
print("="*70)
print("📊 정규화 회귀분석 (Regularized Regression)")
print("="*70)

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/Test 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 모델 비교
models = {
    'OLS': LinearRegression(),
    'Ridge (α=1.0)': Ridge(alpha=1.0),
    'Lasso (α=1.0)': Lasso(alpha=1.0),
    'ElasticNet (α=1.0, l1=0.5)': ElasticNet(alpha=1.0, l1_ratio=0.5)
}

print(f"\n{'모델':<30} {'Train R²':>12} {'Test R²':>12} {'Test MAE(억)':>15}")
print("-"*70)

model_results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    train_r2 = model.score(X_train, y_train)
    test_r2 = model.score(X_test, y_test)
    test_pred = model.predict(X_test)
    test_mae = mean_absolute_error(y_test, test_pred)
    
    model_results[name] = {'model': model, 'train_r2': train_r2, 'test_r2': test_r2, 'mae': test_mae}
    print(f"{name:<30} {train_r2:>12.4f} {test_r2:>12.4f} {test_mae/1e8:>15.2f}")

print("\n※ Ridge/Lasso는 다중공선성이 있거나 과적합이 우려될 때 유용")

📊 정규화 회귀분석 (Regularized Regression)

모델                                 Train R²      Test R²     Test MAE(억)
----------------------------------------------------------------------
OLS                                  0.6782       0.7174           12.67
Ridge (α=1.0)                        0.6782       0.7174           12.67
Lasso (α=1.0)                        0.6782       0.7174           12.67
ElasticNet (α=1.0, l1=0.5)           0.6233       0.6380           12.78

※ Ridge/Lasso는 다중공선성이 있거나 과적합이 우려될 때 유용


In [21]:
# 교차검증 (Cross-Validation)
print("\n" + "="*70)
print("📊 교차검증 (5-Fold Cross Validation)")
print("="*70)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

print(f"\n{'모델':<30} {'CV R² Mean':>12} {'CV R² Std':>12}")
print("-"*55)

for name, model in models.items():
    cv_scores = cross_val_score(model, X_scaled, y, cv=kfold, scoring='r2')
    print(f"{name:<30} {cv_scores.mean():>12.4f} {cv_scores.std():>12.4f}")

print("\n※ CV R² Std가 작을수록 모델이 안정적")


📊 교차검증 (5-Fold Cross Validation)

모델                               CV R² Mean    CV R² Std
-------------------------------------------------------
OLS                                  0.6829       0.0357
Ridge (α=1.0)                        0.6829       0.0357
Lasso (α=1.0)                        0.6829       0.0357
ElasticNet (α=1.0, l1=0.5)           0.6312       0.0243

※ CV R² Std가 작을수록 모델이 안정적


In [22]:
# Bootstrap 신뢰구간
print("\n" + "="*70)
print("📊 Bootstrap 신뢰구간 (95%)")
print("="*70)

def bootstrap_coefficients(X, y, n_bootstrap=1000):
    """Bootstrap으로 회귀계수 신뢰구간 추정"""
    n_samples = len(y)
    n_features = X.shape[1]
    coef_samples = np.zeros((n_bootstrap, n_features))
    
    for i in range(n_bootstrap):
        # 복원추출
        idx = np.random.choice(n_samples, n_samples, replace=True)
        X_boot = X[idx]
        y_boot = y[idx]
        
        # 회귀분석
        model = LinearRegression()
        model.fit(X_boot, y_boot)
        coef_samples[i] = model.coef_
    
    return coef_samples

print("\nBootstrap 계산 중... (n=1000)")
boot_coefs = bootstrap_coefficients(X, y, n_bootstrap=1000)

print(f"\n{'변수':<15} {'계수 추정값':>18} {'95% CI Lower':>15} {'95% CI Upper':>15}")
print("-"*70)

model_ols = LinearRegression()
model_ols.fit(X, y)

for i, name in enumerate(feature_cols):
    coef = model_ols.coef_[i]
    ci_lower = np.percentile(boot_coefs[:, i], 2.5)
    ci_upper = np.percentile(boot_coefs[:, i], 97.5)
    
    # 0이 신뢰구간에 포함되는지 확인
    sig = '' if ci_lower <= 0 <= ci_upper else '*'
    print(f"{name:<15} {coef:>18,.0f} {ci_lower:>15,.0f} {ci_upper:>15,.0f} {sig}")

print("-"*70)
print("※ * = 95% CI가 0을 포함하지 않음 (통계적으로 유의)")


📊 Bootstrap 신뢰구간 (95%)

Bootstrap 계산 중... (n=1000)

변수                          계수 추정값    95% CI Lower    95% CI Upper
----------------------------------------------------------------------
점포_수                   399,192,392     374,422,238     423,445,384 *
총_유동인구_수                       186             161             211 *
발달상권                 1,471,873,945   1,205,799,217   1,758,884,451 *
전통시장                 1,194,138,984   1,012,875,442   1,403,256,182 *
관광특구                 2,719,155,654   2,267,126,262   3,141,162,949 *
미분류                    439,417,000     265,334,389     626,744,277 *
----------------------------------------------------------------------
※ * = 95% CI가 0을 포함하지 않음 (통계적으로 유의)


---

# 6. 머신러닝 모델링 (Random Forest, Gradient Boosting)

In [23]:
# 머신러닝 모델 비교
print("="*70)
print("📊 머신러닝 모델 비교")
print("="*70)

# Train/Test 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ml_models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
}

print(f"\n{'모델':<25} {'Train R²':>10} {'Test R²':>10} {'MAE(억)':>12} {'RMSE(억)':>12}")
print("-"*75)

ml_results = {}
for name, model in ml_models.items():
    model.fit(X_train, y_train)
    
    train_r2 = model.score(X_train, y_train)
    test_pred = model.predict(X_test)
    test_r2 = r2_score(y_test, test_pred)
    test_mae = mean_absolute_error(y_test, test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    
    ml_results[name] = {
        'model': model,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'mae': test_mae,
        'rmse': test_rmse,
        'predictions': test_pred
    }
    
    print(f"{name:<25} {train_r2:>10.4f} {test_r2:>10.4f} {test_mae/1e8:>12.2f} {test_rmse/1e8:>12.2f}")

# 최적 모델 선정
best_model = max(ml_results.items(), key=lambda x: x[1]['test_r2'])
print(f"\n🏆 최적 모델: {best_model[0]} (Test R² = {best_model[1]['test_r2']:.4f})")

📊 머신러닝 모델 비교

모델                          Train R²    Test R²       MAE(억)      RMSE(억)
---------------------------------------------------------------------------
Linear Regression             0.6782     0.7174        12.67        18.77
Ridge                         0.6782     0.7175        12.67        18.77
Random Forest                 0.8882     0.8736         8.61        12.55
Gradient Boosting             0.8752     0.8635         9.12        13.04

🏆 최적 모델: Random Forest (Test R² = 0.8736)


In [24]:
# Feature Importance (Random Forest)
print("\n" + "="*70)
print("📊 변수 중요도 (Feature Importance - Random Forest)")
print("="*70)

rf_model = ml_results['Random Forest']['model']
importances = rf_model.feature_importances_

# 중요도 정렬
importance_df = pd.DataFrame({
    '변수': feature_cols,
    '중요도': importances
}).sort_values('중요도', ascending=False)

print(f"\n{'변수':<15} {'중요도':>12} {'비율(%)':>10}")
print("-"*40)
for _, row in importance_df.iterrows():
    print(f"{row['변수']:<15} {row['중요도']:>12.4f} {row['중요도']*100:>10.1f}%")

# 시각화
import plotly.express as px

fig = px.bar(
    importance_df,
    x='중요도',
    y='변수',
    orientation='h',
    text=importance_df['중요도'].round(3),
    color='중요도',
    color_continuous_scale='RdYlGn',
    title='랜덤 포레스트 변수 중요도'
)

fig.update_layout(
    xaxis_title='변수 중요도',
    yaxis_title='',
    title_font_size=14,
    title_font_family='Malgun Gothic',
    font=dict(
        family='Malgun Gothic',
        size=12
    ),
    coloraxis_showscale=False,
    height=500
)

fig.update_traces(
    textposition='outside'
)

fig.show()



📊 변수 중요도 (Feature Importance - Random Forest)

변수                       중요도      비율(%)
----------------------------------------
점포_수                  0.7516       75.2%
총_유동인구_수              0.1988       19.9%
관광특구                  0.0201        2.0%
발달상권                  0.0184        1.8%
전통시장                  0.0083        0.8%
미분류                   0.0027        0.3%


In [25]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# --------------------------------------------------
# 데이터 준비
# --------------------------------------------------
model_names = list(ml_results.keys())
train_r2s = [ml_results[m]['train_r2'] for m in model_names]
test_r2s  = [ml_results[m]['test_r2']  for m in model_names]

best_model_name = best_model[0]
best_pred = best_model[1]['predictions']

# --------------------------------------------------
# 1행 2열 subplot 생성
# --------------------------------------------------
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        '모델 성능 비교 (R² 점수)',
        f'{best_model_name} : 실제 매출 vs 예측 매출'
    )
)

# --------------------------------------------------
# 1️⃣ R² 비교 (막대그래프)
# --------------------------------------------------
fig.add_trace(
    go.Bar(
        x=model_names,
        y=train_r2s,
        name='Train R²',
        offsetgroup=0
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=model_names,
        y=test_r2s,
        name='Test R²',
        offsetgroup=1
    ),
    row=1, col=1
)

fig.update_yaxes(
    title_text='R²',
    range=[0, 1],
    row=1, col=1
)

# --------------------------------------------------
# 2️⃣ 실제 vs 예측 산점도
# --------------------------------------------------
fig.add_trace(
    go.Scatter(
        x=y_test / 1e8,
        y=best_pred / 1e8,
        mode='markers',
        name='예측값',
        opacity=0.5
    ),
    row=1, col=2
)

max_val = max(y_test.max(), best_pred.max()) / 1e8

# 완벽 예측선 (y=x)
fig.add_trace(
    go.Scatter(
        x=[0, max_val],
        y=[0, max_val],
        mode='lines',
        name='완벽 예측선',
        line=dict(dash='dash')
    ),
    row=1, col=2
)

fig.update_xaxes(
    title_text='실제 매출 (억원)',
    row=1, col=2
)

fig.update_yaxes(
    title_text='예측 매출 (억원)',
    row=1, col=2
)

# --------------------------------------------------
# 전체 레이아웃
# --------------------------------------------------
fig.update_layout(
    height=500,
    width=1100,
    barmode='group',
    font=dict(
        family='Malgun Gothic',
        size=12
    ),
    legend_title_text='구분'
)

fig.show()


---

# 7. What-if 시나리오 분석

In [26]:
# 예측 함수 정의
print("="*70)
print("📊 What-if 시나리오 분석")
print("="*70)

# OLS 모델로 예측 함수 생성
model_final = LinearRegression()
model_final.fit(X, y)

def predict_sales(점포수, 유동인구, 상권유형='골목상권'):
    """
    입지 조건을 입력하면 예상 분기 매출(억원)을 반환
    """
    발달 = 1 if 상권유형 == '발달상권' else 0
    전통 = 1 if 상권유형 == '전통시장' else 0
    관광 = 1 if 상권유형 == '관광특구' else 0
    미분류_v = 1 if 상권유형 == '미분류' else 0
    
    input_data = np.array([[점포수, 유동인구, 발달, 전통, 관광, 미분류_v]])
    pred = model_final.predict(input_data)[0]
    return pred / 1e8

print("\n✅ 예측 함수 생성 완료: predict_sales(점포수, 유동인구, 상권유형)")

📊 What-if 시나리오 분석

✅ 예측 함수 생성 완료: predict_sales(점포수, 유동인구, 상권유형)


In [27]:
# 기준 조건 설정
avg_stores = df['점포_수'].mean()
avg_pop = df['총_유동인구_수'].mean()
base_sales = predict_sales(avg_stores, avg_pop, '골목상권')

print(f"\n[기준 조건]")
print(f"  점포수: {avg_stores:.1f}개")
print(f"  유동인구: {avg_pop/1e6:.2f}백만명")
print(f"  상권유형: 골목상권")
print(f"  기준 매출: {base_sales:.2f}억원")


[기준 조건]
  점포수: 6.2개
  유동인구: 5.62백만명
  상권유형: 골목상권
  기준 매출: 25.29억원


In [28]:
# 시나리오 1: 유동인구 변화에 따른 매출 변화
print("\n" + "="*70)
print("[시나리오 1] 유동인구 변화에 따른 매출 변화")
print("="*70)

pop_changes = [-30, -20, -10, 0, 10, 20, 30, 50, 100]
pop_results = []

print(f"\n{'유동인구 변화':>12} {'새 유동인구(만)':>15} {'예상 매출(억)':>12} {'매출 변화(억)':>12} {'변화율(%)':>10}")
print("-"*70)

for change in pop_changes:
    new_pop = avg_pop * (1 + change/100)
    new_sales = predict_sales(avg_stores, new_pop, '골목상권')
    diff = new_sales - base_sales
    pct = (new_sales / base_sales - 1) * 100
    
    pop_results.append({'변화율': change, '유동인구': new_pop, '매출': new_sales, '차이': diff, '매출변화율': pct})
    print(f"{change:>+10}% {new_pop/1e4:>15,.0f} {new_sales:>12.2f} {diff:>+12.2f} {pct:>+10.1f}%")

print("\n📌 인사이트: 유동인구 10% 증가 시 매출 약 4% 증가")


[시나리오 1] 유동인구 변화에 따른 매출 변화

     유동인구 변화       새 유동인구(만)     예상 매출(억)     매출 변화(억)     변화율(%)
----------------------------------------------------------------------
       -30%             393        22.15        -3.13      -12.4%
       -20%             450        23.20        -2.09       -8.3%
       -10%             506        24.24        -1.04       -4.1%
        +0%             562        25.29        +0.00       +0.0%
       +10%             618        26.33        +1.04       +4.1%
       +20%             674        27.38        +2.09       +8.3%
       +30%             730        28.42        +3.13      +12.4%
       +50%             843        30.51        +5.22      +20.7%
      +100%           1,124        35.74       +10.45      +41.3%

📌 인사이트: 유동인구 10% 증가 시 매출 약 4% 증가


In [29]:
# 시나리오 2: 점포수 변화에 따른 매출 변화
print("\n" + "="*70)
print("[시나리오 2] 점포수 변화에 따른 매출 변화")
print("="*70)

store_changes = [-5, -3, -1, 0, 1, 3, 5, 10]
store_results = []

print(f"\n{'점포수 변화':>10} {'새 점포수':>10} {'예상 매출(억)':>12} {'매출 변화(억)':>12} {'변화율(%)':>10}")
print("-"*60)

for change in store_changes:
    new_stores = max(0, avg_stores + change)
    new_sales = predict_sales(new_stores, avg_pop, '골목상권')
    diff = new_sales - base_sales
    pct = (new_sales / base_sales - 1) * 100 if base_sales != 0 else 0
    
    store_results.append({'변화': change, '점포수': new_stores, '매출': new_sales, '차이': diff, '매출변화율': pct})
    print(f"{change:>+8}개 {new_stores:>10.1f} {new_sales:>12.2f} {diff:>+12.2f} {pct:>+10.1f}%")

print("\n📌 인사이트: 점포 1개 증가 시 매출 약 4억원 증가")


[시나리오 2] 점포수 변화에 따른 매출 변화

    점포수 변화      새 점포수     예상 매출(억)     매출 변화(억)     변화율(%)
------------------------------------------------------------
      -5개        1.2         5.33       -19.96      -78.9%
      -3개        3.2        13.31       -11.98      -47.4%
      -1개        5.2        21.30        -3.99      -15.8%
      +0개        6.2        25.29        +0.00       +0.0%
      +1개        7.2        29.28        +3.99      +15.8%
      +3개        9.2        37.26       +11.98      +47.4%
      +5개       11.2        45.25       +19.96      +78.9%
     +10개       16.2        65.21       +39.92     +157.9%

📌 인사이트: 점포 1개 증가 시 매출 약 4억원 증가


In [30]:
# 시나리오 3: 복합 시나리오 분석
print("\n" + "="*70)
print("[시나리오 3] 복합 시나리오 분석 - 실제 입지 의사결정")
print("="*70)

scenarios = [
    {"name": "현상 유지", "점포": avg_stores, "유동": avg_pop, "상권": "골목상권"},
    {"name": "강남역 발달상권", "점포": 15, "유동": 8_000_000, "상권": "발달상권"},
    {"name": "주택가 소형상권", "점포": 3, "유동": 1_500_000, "상권": "골목상권"},
    {"name": "명동 관광특구", "점포": 25, "유동": 10_000_000, "상권": "관광특구"},
    {"name": "전통시장 인근", "점포": 5, "유동": 3_000_000, "상권": "전통시장"},
    {"name": "신도시 신규개발", "점포": 8, "유동": 5_000_000, "상권": "골목상권"},
]

print(f"\n{'시나리오':<20} {'점포수':>8} {'유동인구(만)':>12} {'상권유형':<10} {'예상매출(억)':>12}")
print("-"*75)

for sc in scenarios:
    sales = predict_sales(sc["점포"], sc["유동"], sc["상권"])
    print(f"{sc['name']:<20} {sc['점포']:>8} {sc['유동']/1e4:>12,.0f} {sc['상권']:<10} {sales:>12.2f}")


[시나리오 3] 복합 시나리오 분석 - 실제 입지 의사결정

시나리오                      점포수      유동인구(만) 상권유형            예상매출(억)
---------------------------------------------------------------------------
현상 유지                6.155650319829424          562 골목상권              25.29
강남역 발달상권                   15          800 발달상권              79.74
주택가 소형상권                    3          150 골목상권               5.03
명동 관광특구                    25        1,000 관광특구             135.85
전통시장 인근                     5          300 전통시장              27.75
신도시 신규개발                    8          500 골목상권              31.50


In [31]:
# 데이터프레임 변환
pop_df = pd.DataFrame(pop_results)
store_df = pd.DataFrame(store_results)

# --------------------------------------------------
# 1행 2열 subplot
# --------------------------------------------------
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        '민감도 분석: 유동인구 변화 효과',
        '민감도 분석: 점포 수 변화 효과'
    )
)

# --------------------------------------------------
# 1️⃣ 유동인구 변화 효과
# --------------------------------------------------
fig.add_trace(
    go.Scatter(
        x=pop_df['변화율'],
        y=pop_df['매출'],
        mode='lines+markers',
        name='예상 매출',
        marker=dict(size=8),
        line=dict(width=2)
    ),
    row=1, col=1
)

# 기준 매출선
fig.add_hline(
    y=base_sales,
    line_dash='dash',
    annotation_text=f'기준 매출: {base_sales:.1f}',
    annotation_position='top left',
    row=1, col=1
)

# 변화율 0 기준선
fig.add_vline(
    x=0,
    line_dash='dot',
    line_color='gray',
    opacity=0.5,
    row=1, col=1
)

fig.update_xaxes(
    title_text='유동인구 변화율 (%)',
    row=1, col=1
)

fig.update_yaxes(
    title_text='예상 매출 (억원)',
    row=1, col=1
)

# --------------------------------------------------
# 2️⃣ 점포 수 변화 효과
# --------------------------------------------------
fig.add_trace(
    go.Scatter(
        x=store_df['변화'],
        y=store_df['매출'],
        mode='lines+markers',
        name='예상 매출',
        marker=dict(size=8),
        line=dict(width=2)
    ),
    row=1, col=2
)

# 기준 매출선
fig.add_hline(
    y=base_sales,
    line_dash='dash',
    annotation_text=f'기준 매출: {base_sales:.1f}',
    annotation_position='top left',
    row=1, col=2
)

# 변화 0 기준선
fig.add_vline(
    x=0,
    line_dash='dot',
    line_color='gray',
    opacity=0.5,
    row=1, col=2
)

fig.update_xaxes(
    title_text='점포 수 변화',
    row=1, col=2
)

fig.update_yaxes(
    title_text='예상 매출 (억원)',
    row=1, col=2
)

# --------------------------------------------------
# 전체 레이아웃
# --------------------------------------------------
fig.update_layout(
    height=500,
    width=1100,
    font=dict(
        family='Malgun Gothic',
        size=12
    ),
    showlegend=False
)

fig.show()


---

# 8. 상권유형 전환 효과 분석

In [32]:
# 상권유형 전환 효과 분석
print("="*70)
print("📊 상권유형 전환 효과 분석")
print("="*70)
print("\n동일한 조건에서 상권유형만 다를 때의 매출 차이 분석")

base_stores = avg_stores
base_pop = avg_pop

print(f"\n[분석 조건]")
print(f"  점포수: {base_stores:.1f}개")
print(f"  유동인구: {base_pop/1e6:.2f}백만명")

area_types_list = ['골목상권', '발달상권', '전통시장', '관광특구', '미분류']
골목_매출 = predict_sales(base_stores, base_pop, '골목상권')

print(f"\n[상권유형별 예상 매출]")
print("-"*60)
print(f"{'상권유형':<12} {'예상 매출(억)':>12} {'골목상권 대비':>15} {'증감율(%)':>12}")
print("-"*60)

상권매출 = {}
for 상권 in area_types_list:
    매출 = predict_sales(base_stores, base_pop, 상권)
    상권매출[상권] = 매출
    차이 = 매출 - 골목_매출
    증감율 = (매출 / 골목_매출 - 1) * 100 if 골목_매출 != 0 else 0
    marker = "(기준)" if 상권 == '골목상권' else ""
    print(f"{상권:<12} {매출:>12.2f} {차이:>+15.2f} {증감율:>+12.1f}% {marker}")

📊 상권유형 전환 효과 분석

동일한 조건에서 상권유형만 다를 때의 매출 차이 분석

[분석 조건]
  점포수: 6.2개
  유동인구: 5.62백만명

[상권유형별 예상 매출]
------------------------------------------------------------
상권유형             예상 매출(억)         골목상권 대비       증감율(%)
------------------------------------------------------------
골목상권                25.29           +0.00         +0.0% (기준)
발달상권                40.01          +14.72        +58.2% 
전통시장                37.23          +11.94        +47.2% 
관광특구                52.48          +27.19       +107.5% 
미분류                 29.68           +4.39        +17.4% 


In [33]:
# 상권유형별 전환 효과 매트릭스
print("\n" + "="*70)
print("📊 상권유형 전환 효과 매트릭스 (단위: 억원)")
print("="*70)
print("\n행: 현재 상권유형, 열: 전환 후 상권유형")
print("값: 전환 시 매출 변화 (양수=매출 증가, 음수=매출 감소)")

print(f"\n{'FROM \\ TO':<12}", end='')
for to_type in area_types_list:
    print(f"{to_type:<12}", end='')
print()
print("-"*75)

for from_type in area_types_list:
    from_sales = 상권매출[from_type]
    print(f"{from_type:<12}", end='')
    for to_type in area_types_list:
        to_sales = 상권매출[to_type]
        diff = to_sales - from_sales
        if from_type == to_type:
            print(f"{'---':>12}", end='')
        else:
            print(f"{diff:>+12.1f}", end='')
    print()

print("\n📌 핵심 인사이트:")
print(f"  • 골목상권 → 관광특구 전환 시: +{상권매출['관광특구']-상권매출['골목상권']:.1f}억원 (가장 큰 효과)")
print(f"  • 골목상권 → 발달상권 전환 시: +{상권매출['발달상권']-상권매출['골목상권']:.1f}억원")
print(f"  • 골목상권 → 전통시장 전환 시: +{상권매출['전통시장']-상권매출['골목상권']:.1f}억원")


📊 상권유형 전환 효과 매트릭스 (단위: 억원)

행: 현재 상권유형, 열: 전환 후 상권유형
값: 전환 시 매출 변화 (양수=매출 증가, 음수=매출 감소)

FROM \ TO   골목상권        발달상권        전통시장        관광특구        미분류         
---------------------------------------------------------------------------
골목상권                 ---       +14.7       +11.9       +27.2        +4.4
발달상권               -14.7         ---        -2.8       +12.5       -10.3
전통시장               -11.9        +2.8         ---       +15.3        -7.5
관광특구               -27.2       -12.5       -15.3         ---       -22.8
미분류                 -4.4       +10.3        +7.5       +22.8         ---

📌 핵심 인사이트:
  • 골목상권 → 관광특구 전환 시: +27.2억원 (가장 큰 효과)
  • 골목상권 → 발달상권 전환 시: +14.7억원
  • 골목상권 → 전통시장 전환 시: +11.9억원


In [34]:
import plotly.graph_objects as go

# 색상 매핑
colors = {
    '골목상권': '#4E79A7',
    '발달상권': '#F28E2B',
    '전통시장': '#59A14F',
    '관광특구': '#E15759',
    '미분류': '#76B7B2'
}

# 상권 순서 (존재하는 것만)
상권순서 = ['골목상권', '미분류', '전통시장', '발달상권', '관광특구']
상권순서 = [s for s in 상권순서 if s in 상권매출]

# 값 및 라벨 생성
y_vals = [상권매출[s] for s in 상권순서]
bar_colors = [colors.get(s, 'gray') for s in 상권순서]

labels = []
for s, v in zip(상권순서, y_vals):
    diff = v - 골목_매출
    if s == '골목상권':
        labels.append(f'{v:.1f}')
    else:
        labels.append(f'{v:.1f}<br>({diff:+.1f})')

# --------------------------------------------------
# Plotly Bar Chart
# --------------------------------------------------
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=상권순서,
        y=y_vals,
        marker_color=bar_colors,
        text=labels,
        textposition='outside'
    )
)

fig.update_layout(
    title=dict(
        text='상권유형 전환 효과<br><span style="font-size:12px">(동일 조건: 점포 수 & 유동인구)</span>',
        x=0.5
    ),
    yaxis_title='예상 매출 (억원)',
    font=dict(
        family='Malgun Gothic',
        size=12
    ),
    height=500,
    showlegend=False
)

fig.update_yaxes(
    showgrid=True,
    gridcolor='rgba(0,0,0,0.15)'
)

fig.show()


---

# 9. 종합 예측 및 최종 결론

In [35]:
# 예측 모델 종합 요약
print("="*80)
print("📊 예측 모델 종합 요약")
print("="*80)

# 최종 모델 계수
print(f"\n[최종 예측 모델: 다중선형회귀]")
print(f"\n  매출(억원) = {model_final.intercept_/1e8:.2f}")
print(f"             + {model_final.coef_[0]/1e8:.2f} × 점포수")
print(f"             + {model_final.coef_[1]*1e6/1e8:.2f} × 유동인구(백만명)")
print(f"             + {model_final.coef_[2]/1e8:.2f} × 발달상권(더미)")
print(f"             + {model_final.coef_[3]/1e8:.2f} × 전통시장(더미)")
print(f"             + {model_final.coef_[4]/1e8:.2f} × 관광특구(더미)")
print(f"             + {model_final.coef_[5]/1e8:.2f} × 미분류(더미)")

print(f"\n[모델 성능 지표]")
print(f"  R² (설명력): {R2:.4f} ({R2*100:.1f}%)")
print(f"  Adjusted R²: {R2_adj:.4f}")
print(f"  RMSE: {RMSE/1e8:.2f}억원")
print(f"  F-통계량: {F_stat:.2f} (p < 0.001)")

📊 예측 모델 종합 요약

[최종 예측 모델: 다중선형회귀]

  매출(억원) = -9.73
             + 3.99 × 점포수
             + 1.86 × 유동인구(백만명)
             + 14.72 × 발달상권(더미)
             + 11.94 × 전통시장(더미)
             + 27.19 × 관광특구(더미)
             + 4.39 × 미분류(더미)

[모델 성능 지표]
  R² (설명력): 0.6881 (68.8%)
  Adjusted R²: 0.6878
  RMSE: 17.89억원
  F-통계량: 2238.95 (p < 0.001)


In [36]:
# 가설 검증 결과 종합
print("\n" + "="*80)
print("📊 가설 검증 결과 종합")
print("="*80)

print(f"""
┌──────────────────────────────────────────────────────────────────────────┐
│                           가설 검증 결과                                 │
├──────────────────────────────────────────────────────────────────────────┤
│ H1: 유동인구가 많을수록 매출이 높다                                       │
│     → ✅ 채택 (β={model_final.coef_[1]:.2f}, p<0.001)                            │
│     → 유동인구 100만명 증가 시 매출 약 {model_final.coef_[1]*1e6/1e8:.2f}억원 증가                │
├──────────────────────────────────────────────────────────────────────────┤
│ H2: 점포수(밀집도)가 높을수록 매출이 높다                                 │
│     → ✅ 채택 (β={model_final.coef_[0]/1e8:.2f}억, p<0.001)                           │
│     → 점포 1개 증가 시 매출 약 {model_final.coef_[0]/1e8:.2f}억원 증가 (가장 강력한 효과)    │
├──────────────────────────────────────────────────────────────────────────┤
│ H3: 상권유형에 따라 매출 차이가 있다                                      │
│     → ✅ 채택 (ANOVA F={f_stat:.2f}, p<0.001)                              │
│     → 관광특구 > 발달상권 > 전통시장 > 골목상권 순으로 높은 매출          │
└──────────────────────────────────────────────────────────────────────────┘
""")


📊 가설 검증 결과 종합

┌──────────────────────────────────────────────────────────────────────────┐
│                           가설 검증 결과                                 │
├──────────────────────────────────────────────────────────────────────────┤
│ H1: 유동인구가 많을수록 매출이 높다                                       │
│     → ✅ 채택 (β=185.94, p<0.001)                            │
│     → 유동인구 100만명 증가 시 매출 약 1.86억원 증가                │
├──────────────────────────────────────────────────────────────────────────┤
│ H2: 점포수(밀집도)가 높을수록 매출이 높다                                 │
│     → ✅ 채택 (β=3.99억, p<0.001)                           │
│     → 점포 1개 증가 시 매출 약 3.99억원 증가 (가장 강력한 효과)    │
├──────────────────────────────────────────────────────────────────────────┤
│ H3: 상권유형에 따라 매출 차이가 있다                                      │
│     → ✅ 채택 (ANOVA F=206.46, p<0.001)                              │
│     → 관광특구 > 발달상권 > 전통시장 > 골목상권 순으로 높은 매출          │
└────────────────────────────────────────────────────────────

In [37]:
# 저평가/고평가 지역 분석
print("\n" + "="*80)
print("📊 저평가/고평가 지역 분석 (2025년 3분기 기준)")
print("="*80)

# 예측값 계산
df['예측_매출'] = model_final.predict(X)
df['잔차'] = df['당월_매출_금액'] - df['예측_매출']
df['잔차율'] = df['잔차'] / df['예측_매출'] * 100

# 최근 분기
latest_qtr = df['기준_년분기_코드'].max()
df_latest = df[df['기준_년분기_코드'] == latest_qtr].copy()

print(f"\n[저평가 지역 TOP 10] (예측 > 실제 → 잠재력 있는 지역)")
print("-"*75)
underperform = df_latest.nsmallest(10, '잔차')[['행정동_코드_명', '주요_상권유형', '점포_수', '총_유동인구_수', '당월_매출_금액', '예측_매출', '잔차']]
print(f"{'행정동':<12} {'상권':8} {'점포':>4} {'유동인구(만)':>10} {'실제(억)':>8} {'예측(억)':>8} {'차이(억)':>8}")
print("-"*75)
for _, row in underperform.iterrows():
    print(f"{row['행정동_코드_명']:<12} {row['주요_상권유형']:<8} {row['점포_수']:>4.0f} {row['총_유동인구_수']/1e4:>10.0f} {row['당월_매출_금액']/1e8:>8.1f} {row['예측_매출']/1e8:>8.1f} {row['잔차']/1e8:>+8.1f}")

print(f"\n[고평가 지역 TOP 10] (예측 < 실제 → 벤치마킹 대상)")
print("-"*75)
outperform = df_latest.nlargest(10, '잔차')[['행정동_코드_명', '주요_상권유형', '점포_수', '총_유동인구_수', '당월_매출_금액', '예측_매출', '잔차']]
print(f"{'행정동':<12} {'상권':<8} {'점포':>4} {'유동인구(만)':>10} {'실제(억)':>8} {'예측(억)':>8} {'차이(억)':>8}")
print("-"*75)
for _, row in outperform.iterrows():
    print(f"{row['행정동_코드_명']:<12} {row['주요_상권유형']:<8} {row['점포_수']:>4.0f} {row['총_유동인구_수']/1e4:>10.0f} {row['당월_매출_금액']/1e8:>8.1f} {row['예측_매출']/1e8:>8.1f} {row['잔차']/1e8:>+8.1f}")


📊 저평가/고평가 지역 분석 (2025년 3분기 기준)

[저평가 지역 TOP 10] (예측 > 실제 → 잠재력 있는 지역)
---------------------------------------------------------------------------
행정동          상권         점포    유동인구(만)    실제(억)    예측(억)    차이(억)
---------------------------------------------------------------------------
강일동          골목상권       14        351      7.9     52.7    -44.8
상봉1동         골목상권       13        476      7.7     51.0    -43.3
신도림동         골목상권       13        648     14.1     54.2    -40.1
공덕동          골목상권        9       1219     11.4     48.9    -37.5
공항동          골목상권       13        541     15.0     52.2    -37.2
연희동          골목상권       14        870     28.8     62.3    -33.5
월계1동         골목상권       13        450     18.2     50.5    -32.3
구로2동         골목상권       10        882     14.5     46.6    -32.1
화곡본동         골목상권       10        821     13.8     45.5    -31.6
신정4동         골목상권       17       1064     46.3     77.9    -31.6

[고평가 지역 TOP 10] (예측 < 실제 → 벤치마킹 대상)
-------------------------

---

# 10. 핵심 인사이트 및 전략적 제언

In [38]:
# 핵심 인사이트 종합
print("="*80)
print("🎯 핵심 인사이트 및 전략적 제언")
print("="*80)

print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                          📌 핵심 발견사항 (Key Findings)                      ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  1️⃣  점포 수가 매출의 가장 강력한 결정요인                                    ║
║      • 점포 1개 증가 → 매출 약 4억원 증가                                     ║
║      • 점포 밀집도가 높은 지역 = 수요가 검증된 상권                           ║
║                                                                              ║
║  2️⃣  상권유형이 매출 수준을 크게 좌우                                         ║
║      • 관광특구: 골목상권 대비 +27억원 (가장 높음)                            ║
║      • 발달상권: 골목상권 대비 +15억원                                        ║
║      • 전통시장: 골목상권 대비 +12억원                                        ║
║                                                                              ║
║  3️⃣  유동인구 효과는 존재하나 상대적으로 제한적                               ║
║      • 유동인구 100만명 증가 → 매출 약 1.9억원 증가                           ║
║      • 단순 유동인구보다 상권 특성이 더 중요                                  ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")

🎯 핵심 인사이트 및 전략적 제언

╔══════════════════════════════════════════════════════════════════════════════╗
║                          📌 핵심 발견사항 (Key Findings)                      ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  1️⃣  점포 수가 매출의 가장 강력한 결정요인                                    ║
║      • 점포 1개 증가 → 매출 약 4억원 증가                                     ║
║      • 점포 밀집도가 높은 지역 = 수요가 검증된 상권                           ║
║                                                                              ║
║  2️⃣  상권유형이 매출 수준을 크게 좌우                                         ║
║      • 관광특구: 골목상권 대비 +27억원 (가장 높음)                            ║
║      • 발달상권: 골목상권 대비 +15억원                                        ║
║      • 전통시장: 골목상권 대비 +12억원                                        ║
║                                                                              ║
║  3️⃣  유동인구 효과는 존재하나 상대

In [39]:
# 전략적 제언
print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                          💡 전략적 제언 (Recommendations)                    ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  🏪 신규 출점 전략                                                           ║
║  ────────────────────────────────────────────────────────────                ║
║  • 관광특구/발달상권 우선 검토 (높은 기대 매출)                               ║
║  • 기존 점포 밀집 지역 인근 출점 고려 (검증된 수요)                           ║
║  • 저평가 지역 집중 공략 (잠재 성장 가능성)                                   ║
║                                                                              ║
║  📈 매출 증대 전략                                                           ║
║  ────────────────────────────────────────────────────────────                ║
║  • 고평가 지역(벤치마킹 대상) 성공 요인 분석                                  ║
║  • 유동인구 증가 시기(이벤트, 계절) 타겟 프로모션                             ║
║  • 상권 특성에 맞는 상품 구성 차별화                                          ║
║                                                                              ║
║  ⚠️  주의사항                                                                ║
║  ────────────────────────────────────────────────────────────                ║
║  • 단순 유동인구만으로 입지 선정 금지                                         ║
║  • 상권유형별 경쟁 강도 고려 필요                                             ║
║  • 모델 예측은 평균적 경향 → 개별 입지 특성 추가 검토 필수                    ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")


╔══════════════════════════════════════════════════════════════════════════════╗
║                          💡 전략적 제언 (Recommendations)                    ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  🏪 신규 출점 전략                                                           ║
║  ────────────────────────────────────────────────────────────                ║
║  • 관광특구/발달상권 우선 검토 (높은 기대 매출)                               ║
║  • 기존 점포 밀집 지역 인근 출점 고려 (검증된 수요)                           ║
║  • 저평가 지역 집중 공략 (잠재 성장 가능성)                                   ║
║                                                                              ║
║  📈 매출 증대 전략                                                           ║
║  ────────────────────────────────────────────────────────────                ║
║  • 고평가 지역(벤치마킹 대상) 성공 요인 분석                                  ║
║  • 유동인구 증가 시기(이벤트, 계절) 타겟 프로모션

In [40]:
# 최종 결론
print("\n" + "="*80)
print("📋 최종 결론 (Executive Summary)")
print("="*80)

print(f"""
┌──────────────────────────────────────────────────────────────────────────────┐
│                                                                              │
│  본 연구는 서울시 편의점 매출 결정요인을 분석한 결과, 다음과 같은 결론을      │
│  도출하였습니다.                                                             │
│                                                                              │
│  📊 분석 결과 요약                                                           │
│  ─────────────────────────────────────────────────────────────────           │
│  • 분석 데이터: 서울시 {len(df['행정동_코드_명'].unique())}개 행정동, {df['기준_년분기_코드'].nunique()}개 분기 ({len(df):,}건)                │
│  • 모델 설명력: R² = {R2:.1%} (매출 변동의 {R2*100:.1f}% 설명)                          │
│  • 모든 독립변수가 통계적으로 유의 (p < 0.001)                                │
│                                                                              │
│  🔑 핵심 메시지                                                              │
│  ─────────────────────────────────────────────────────────────────           │
│                                                                              │
│  "편의점 매출은 점포 밀집도와 상권유형에 의해 주로 결정되며,                  │
│   단순 유동인구보다 상권의 질적 특성이 더 중요하다.                           │
│   관광특구와 발달상권에서 가장 높은 매출이 기대되며,                          │
│   신규 출점 시 점포 밀집도가 높은 검증된 상권을 우선 고려해야 한다."          │
│                                                                              │
│  📅 분석 기간: 2022년 1분기 ~ 2025년 3분기                                    │
│  📝 작성일: 2026년 1월 27일                                                  │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘
""")

print("\n" + "="*80)
print("✅ 분석 완료")
print("="*80)


📋 최종 결론 (Executive Summary)

┌──────────────────────────────────────────────────────────────────────────────┐
│                                                                              │
│  본 연구는 서울시 편의점 매출 결정요인을 분석한 결과, 다음과 같은 결론을      │
│  도출하였습니다.                                                             │
│                                                                              │
│  📊 분석 결과 요약                                                           │
│  ─────────────────────────────────────────────────────────────────           │
│  • 분석 데이터: 서울시 413개 행정동, 15개 분기 (6,097건)                │
│  • 모델 설명력: R² = 68.8% (매출 변동의 68.8% 설명)                          │
│  • 모든 독립변수가 통계적으로 유의 (p < 0.001)                                │
│                                                                              │
│  🔑 핵심 메시지                                                              │
│  ─────────────────────────────────────────────────────────────────           │
│           